# VacationPy
Using the weather and city data from WeatherPy, create heat maps and other graphs to help plan your next vacation!

In [1]:
# import dependences 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

In [2]:
# read in the city csv from WeatherPy

cities = pd.read_csv("burns_cities.csv")
cities.head()

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yunyang,33.45,112.71,55.26,38,0,5.64,CN,1604785852
1,1,salalah,17.02,54.09,75.20,64,0,2.24,OM,1604785295
2,2,karratha,-20.74,116.85,70.59,64,0,9.93,AU,1604785594
3,3,ushuaia,-54.80,-68.30,59.00,44,75,12.75,AR,1604785272
4,4,college,64.86,-147.80,27.00,79,90,4.70,US,1604785474


# Heat Map
Create a heat map that displays the humidity for every city from Part I

In [3]:
humidity = cities["Humidity"].astype(float)
max_humidity = humidity.max()

print(max_humidity)
city_locations = cities[["Latitude", "Longitude"]]

100.0


In [4]:
# Configure gmaps with API key
city_locations = cities[["Latitude", "Longitude"]]
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity,
                                 max_intensity=max_humidity,
                                 dissipating=False,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Narrow Down the Data Frame
Narrow down the DataFrame to find your ideal weather condition. For example:

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [5]:
clean_city_df = cities.loc[(cities["Temperature"] > 70) &
                                          (cities["Temperature"] < 80) &
                                          (cities["Wind Speed"] < 10) &
                                          (cities["Cloudiness"] > 5)
                                          , :]
clean_city_df = clean_city_df.dropna(how='any')
clean_city_df.reset_index(inplace=True)
del clean_city_df['index']

# Display to output
clean_city_df.head()

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,5,arraial do cabo,-22.97,-42.03,73.40,78,75,5.82,BR,1604785467
1,6,hilo,19.73,-155.09,78.80,78,90,6.93,US,1604785472
2,26,bengkulu,-3.80,102.27,75.74,85,88,4.00,ID,1604785855
3,30,georgetown,5.41,100.34,78.80,94,20,3.47,MY,1604785540
4,41,avarua,-21.21,-159.78,77.00,73,89,8.43,CK,1604785481


# Find and Plot Hotels

Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [6]:
h_list = []

for i in range(len(clean_city_df)):
    lat = clean_city_df.loc[i]['Latitude']
    lng = clean_city_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_list.append(jsn['results'][0]['name'])
    except:
        h_list.append("")

# Append Hotel Name to the selected clean city data frame
clean_city_df["Hotel Name"] = h_list
clean_city_df = clean_city_df.dropna(how='any')
clean_city_df.head()

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,5,arraial do cabo,-22.97,-42.03,73.40,78,75,5.82,BR,1604785467,
1,6,hilo,19.73,-155.09,78.80,78,90,6.93,US,1604785472,
2,26,bengkulu,-3.80,102.27,75.74,85,88,4.00,ID,1604785855,
3,30,georgetown,5.41,100.34,78.80,94,20,3.47,MY,1604785540,
4,41,avarua,-21.21,-159.78,77.00,73,89,8.43,CK,1604785481,


In [7]:
# Create hotel subset data set

clean_city_hotel_df = clean_city_df[["Hotel Name", "City"]]

clean_city_hotel_df.head()

,Hotel Name,City
0,,arraial do cabo
1,,hilo
2,,bengkulu
3,,georgetown
4,,avarua


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_city_df.iterrows()]
locations = clean_city_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))